In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install cellpose==3.0.8

In [ ]:
FOLDER_NAME = "2024.05.09. Ac-kPHSRN+fibronektin-few-annotated"

!unzip "/content/drive/MyDrive/szakmai gyakorlat/cellpose-cell-counting/datasets/{FOLDER_NAME}.zip" -d "/content/{FOLDER_NAME}"

In [ ]:
import numpy as np
import glob
import cv2
from torch.utils.data import Dataset, random_split
from PIL import Image

file_names = glob.glob(FOLDER_NAME + "/*.jpg")

class ImageDataset(Dataset):
  def __len__(self):
    return len(file_names)

  def __getitem__(self, idx):
    img = cv2.imread(file_names[idx], 0)
    mask = Image.open(file_names[idx].replace(".jpg", "_mask.tiff"))
    return img, np.array(mask)

dataset = ImageDataset()

In [ ]:
training_images, training_masks = [], []

for img, mask in dataset:
  training_images.append(img)
  training_masks.append(mask)

In [ ]:
from cellpose import models, train
from tqdm import trange
import torch
import logging

def train_model(model_type, n_epochs):
  logger = logging.getLogger('cellpose')
  logger.setLevel(logging.DEBUG)

  model = models.CellposeModel(model_type=model_type, gpu=torch.cuda.is_available())

  train.train_seg(
              model.net, n_epochs=n_epochs,
              train_data=training_images, train_labels=training_masks,
              channels=[0, 0], normalize=True,
              weight_decay=1e-4, SGD=True, learning_rate=0.1,
              save_path="/content/drive/MyDrive/szakmai gyakorlat/cellpose-cell-counting", model_name=f"{model_type}_new_annotated")

In [ ]:
train_model("cyto3", n_epochs = 1000)

INFO:cellpose.models:>> cyto3 << model set to be used
INFO:cellpose.models:Downloading: "https://www.cellpose.org/models/cyto3" to /root/.cellpose/models/cyto3

100%|██████████| 25.3M/25.3M [00:03<00:00, 8.78MB/s]
INFO:cellpose.core:** TORCH CUDA version installed and working. **
INFO:cellpose.core:>>>> using GPU
INFO:cellpose.models:>>>> loading model /root/.cellpose/models/cyto3
INFO:cellpose.models:>>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
INFO:cellpose.dynamics:computing flows for labels
100%|██████████| 11/11 [00:02<00:00,  4.75it/s]
INFO:cellpose.train:>>> computing diameters
100%|██████████| 11/11 [00:00<00:00, 1079.72it/s]
INFO:cellpose.train:>>> using channels [0, 0]
INFO:cellpose.train:>>> normalizing {'lowhigh': None, 'percentile': None, 'normalize': True, 'norm3D': False, 'sharpen_radius': 0, 'smooth_radius': 0, 'tile_norm_blocksize': 0, 'tile_norm_smooth3D': 1, 'invert': False}
INFO:cellpose.train:>>> n_epochs=1000, n_train=11, n_test=None